## Extract features

Loading our trained network features and weights.
Please make sure:
 - You replace the 'saved_modal_name' to the name of your trained model
 - The data_dir to the directory in your computer that containes the images.
 - Do not forget to set the last layer of your model

In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

data_dir = r'C:\Users\yonba\OneDrive\Desktop\Education\Semester_A_2019\Image_Processing\PhotosDataSet\_Data'

base_model = load_model('../Transfer Lerning ResNet/resnet_exp1_1551432249.h5')

x=base_model.get_layer('activation_48').output
vector=Flatten()(x)

model = Model(inputs=base_model.input, outputs=vector)

datagen = ImageDataGenerator()
generator = datagen.flow_from_directory(
        data_dir,
        target_size=(128, 128),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)

Found 35416 images belonging to 100 classes.


## Creating the data set for the prediction

We want to create 3 lists:
- original images : List of teh original images
- processed images: For each image we will receive an array of predictions that defiens what is the probabilty that the image is of the given class

In [2]:
import numpy as np

imgsList = []
procImgs = []
labels = []
for i in range(generator.n):
    (img, lbl) = generator.next();
    imgsList.append(img)            # Adding the original image to the list
    procImgs.append(np.reshape(model.predict_on_batch(img), -1)) # Using our model to give the images labels
    labels.append(np.argmax(lbl)) # Saving the label with  the highest prediction for each image. 
    
print(labels)

OSError: image file is truncated (13 bytes not processed)

## Training a KNN network

In order to see how good our network is,
we will train a knn network that will use the feature extraction we did with the deep learning network to classify each image.
and then the KNN network will return the X closet neighbors and display them

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neighbors_count = 4

neigh = KNeighborsClassifier(n_neighbors=neighbors_count) # setting the KNN clasifier

neigh.fit(proccImgs, labels) # Training the KNN

## Preprocessing the testing data

 Loading our test data.
 Taking a random image and getting its features with the model we trained earlier.

Please Make sure to change the validation_data_dir to the wanted location.

In [ ]:

test_datagen = ImageDataGenerator()
validation_data_dir = r'C:\Users\yonba\OneDrive\Desktop\Education\Semester_A_2019\Image_Processing\PhotosDataSet\_Test

validation_generator = test_datagen.flow_from_directory(
                                        validation_data_dir,
                                         target_size=(128, 128),
                                         batch_size=1,
                                         color_mode='rgb',
                                        class_mode = "categorical",
                                        shuffle=True)
x_img, y_label = next(validation_generator);
features = model.predict(x_img)

## Display the image

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 20))
plt.subplot(1,1,1)
image = x_img
image= image.astype(int)
image = np.reshape(image, (128, 128, 3))
plt.imshow(image)

## Classify the closest pictures with the KNN algorithm

Taking the most closest pictures to our x_img pic

In [ ]:
labels = neigh.kneighbors(features, return_distance=False) 

print(labels[0])plt.figure(figsize=(20, 20))

for i in range(neighbors_count):
    plt.subplot(2,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image = imgsList[labels[0][i]]
    image= image.astype(int)
    image = np.reshape(image, (128, 128, 3))
    plt.imshow(image)